In [2]:
#To increase cell width of ipynb
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import gzip
import cv2 as cv
import os
import numpy as np

/home/sudhakaran/Desktop/3d_recognition_thesis/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sudhakaran/Desktop/3d_recognition_thesis/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sudhakaran/Desktop/3d_recognition_thesis/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)]

In [6]:
IMG_SIZE = 128
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
save_model_path = '../Saved_Models/custom_MobileNetV2.h5'

In [7]:
def extract_data():
    
    train = []
    test = []
    train_labels = []
    test_labels = []
    label_names = []
    
    data_path = '../ModelNet10/image_dataset'
    
    print("\n..........loading dataset from disk..........\n")

    for label in sorted(os.listdir(data_path)):
        label_path = os.path.join(data_path, label)

        for file in (os.listdir(label_path)):
            file_path = os.path.join(label_path, file)

            if str(file) == "train":
                for instance in sorted(os.listdir(file_path)):
                    instance_path = os.path.join(file_path, instance)
                    train_views = []

                    for img in sorted(os.listdir(instance_path)):
                        img_path = os.path.join(instance_path, img)
                        image = cv.imread(img_path, cv.IMREAD_GRAYSCALE)
                        re_image = cv.resize(image, (IMG_SIZE, IMG_SIZE))
                        train_views.append(re_image)
                    image1 = cv.merge(train_views)
                    train.append(image1)
                    train_labels.append(label)
    #                 cv.imshow("image", image)
    #                 cv.waitKey(0)
    #                 cv.destroyAllWindows()
    #                 cv.imwrite("./2.jpg", image1)
    #                 break

            elif str(file) == "test":
                for instance in sorted(os.listdir(file_path)):
                    instance_path = os.path.join(file_path, instance)
                    test_views = []

                    for img in sorted(os.listdir(instance_path)):
                        img_path = os.path.join(instance_path, img)
                        image = cv.imread(img_path, cv.IMREAD_GRAYSCALE)
                        re_image = cv.resize(image, (IMG_SIZE, IMG_SIZE))
                        test_views.append(re_image)
                    image2 = cv.merge(test_views)
                    test.append(image2)
                    test_labels.append(label)
                    
    train = np.array(train, dtype="float") / 255.0
    train_labels = np.array(train_labels)
    
    test = np.array(test, dtype="float") / 255.0
    test_labels = np.array(test_labels)
    
    label_names = np.unique(train_labels)
    
    for i, y in enumerate(train_labels):
        index = np.searchsorted(label_names, y)
        train_labels[i] = index

    for i, y in enumerate(test_labels):
        index = np.searchsorted(label_names, y)
        test_labels[i] = index

#     train_labels = tf.one_hot(indices=train_labels, depth=len(label_names))
#     test_labels = tf.one_hot(indices=test_labels, depth=len(label_names))

    train_labels = tf.keras.utils.to_categorical(train_labels)
    test_labels = tf.keras.utils.to_categorical(test_labels)
    
    return train, train_labels, test, test_labels, label_names

In [19]:
def train_model():
    
    mobilenet = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, weights=None, include_top=False)
    mobilenet.trainable = True
    
    model = tf.keras.Sequential()
    model.add(mobilenet)
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy'],
    )
    
    model.fit(x=train, y=train_labels, epochs=50, batch_size=10)
    _, accuracy = model.evaluate(test, test_labels)

    model._layers.pop(-1)
    model.save(save_model_path)

    return accuracy

In [20]:
if __name__ == "__main__":
    
    train, train_labels, test, test_labels, label_names = extract_data()
    a = train_model()


..........loading dataset from disk..........

Epoch 1/50
3991/3991 [==============================] - 28s 7ms/sample - loss: 2.1060 - acc: 0.6555
Epoch 2/50
3991/3991 [==============================] - 20s 5ms/sample - loss: 1.0739 - acc: 0.8173
Epoch 3/50
3991/3991 [==============================] - 20s 5ms/sample - loss: 0.9628 - acc: 0.8043
Epoch 4/50
3991/3991 [==============================] - 20s 5ms/sample - loss: 0.2859 - acc: 0.9095
Epoch 5/50
3991/3991 [==============================] - 20s 5ms/sample - loss: 0.3266 - acc: 0.9048
Epoch 6/50
3991/3991 [==============================] - 20s 5ms/sample - loss: 0.2135 - acc: 0.9283
Epoch 7/50
3991/3991 [==============================] - 20s 5ms/sample - loss: 0.2093 - acc: 0.9311
Epoch 8/50
3991/3991 [==============================] - 24s 6ms/sample - loss: 0.3769 - acc: 0.8887
Epoch 9/50
3991/3991 [==============================] - 20s 5ms/sample - loss: 0.4114 - acc: 0.8895
Epoch 10/50
3991/3991 [=============================